# نمایش خروجی ها به صورت گراف

In [2]:
from py2neo import Graph
graph = Graph(password="Zblock.IR")

In [3]:

A = graph.data("MATCH (block :block)<-[:inc]-(tx :tx) WHERE block.height=57043 RETURN tx")
# Finds all non-coinbase transactions in the first 1000 blocks
B = graph.data("MATCH (block:block)-[:inc]-(tx:tx) WHERE block.height < 1000  AND not exists((tx)<-[:in]-(:coinbase))  RETURN block.height AS block, tx.txid AS transaction, size((tx)<-[:in]-()) AS inputs, size((tx)-[:out]->()) AS outputs, tx.fee / tx.size AS fee_satoshis_per_byte ORDER BY block")
# This query allows you to see if two address are connected by a series of transactions. It also returns a path if there are common intermediary addresses.
C = graph.data("MATCH (start :address {address:'17SkEw2md5avVNyYgj6RiXuQKNwkXaxFyQ'}), (end :address {address:'1XPTgDRhN8RFnzniWCddobD9iKZatrvH4'}) MATCH path=shortestPath( (start)-[:in|:out|:locked*]-(end) ) RETURN path")

In [19]:
C

[{'path': (ee42159)<-[:locked]-(f061500)<-[:out {vout:0}]-(f5320b6)<-[:in {scriptSig:"47304402203d39f56560c75f8bc63eef075781550c5406559d1a1822b236e58a1983600a3d02204d329b5dc23a03c4d79311020818fb6a5fea904b4255815d213c3d72f1a2552701410434417dd8d89deaf0f6481c2c160d6de0921624ef7b956f38eef9ed4a64e36877be84b77cdee5a8d92b7d93694f89c3011bf1cbdf4fd7d8ca13b58a7bb4ab0804",sequence:"4294967295",vin:121}]-(e98e85b)-[:locked]->(e12be02)}]

In [21]:
%load_ext py_d3

In [22]:
%%d3 
<style>

.links line {
  stroke: #999;
  stroke-opacity: 0.6;
}

.nodes circle {
  stroke: #fff;
  stroke-width: 1.5px;
}

</style>

<svg width="960" height="600"></svg>

<script>

var svg = d3.select("svg"),
    width = +svg.attr("width"),
    height = +svg.attr("height");

var color = d3.scaleOrdinal(d3.schemeCategory20);

var simulation = d3.forceSimulation()
    .force("link", d3.forceLink().id(function(d) { return d.id; }))
    .force("charge", d3.forceManyBody())
    .force("center", d3.forceCenter(width / 2, height / 2));

d3.json("./Visualization/miserables.json", function(error, graph) {
  if (error) throw error;

  var link = svg.append("g")
      .attr("class", "links")
    .selectAll("line")
    .data(graph.links)
    .enter().append("line")
      .attr("stroke-width", function(d) { return Math.sqrt(d.value); });

  var node = svg.append("g")
      .attr("class", "nodes")
    .selectAll("circle")
    .data(graph.nodes)
    .enter().append("circle")
      .attr("r", 5)
      .attr("fill", function(d) { return color(d.group); })
      .call(d3.drag()
          .on("start", dragstarted)
          .on("drag", dragged)
          .on("end", dragended));

  node.append("title")
      .text(function(d) { return d.id; });

  simulation
      .nodes(graph.nodes)
      .on("tick", ticked);

  simulation.force("link")
      .links(graph.links);

  function ticked() {
    link
        .attr("x1", function(d) { return d.source.x; })
        .attr("y1", function(d) { return d.source.y; })
        .attr("x2", function(d) { return d.target.x; })
        .attr("y2", function(d) { return d.target.y; });

    node
        .attr("cx", function(d) { return d.x; })
        .attr("cy", function(d) { return d.y; });
  }
});

function dragstarted(d) {
  if (!d3.event.active) simulation.alphaTarget(0.3).restart();
  d.fx = d.x;
  d.fy = d.y;
}

function dragged(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

function dragended(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  d.fx = null;
  d.fy = null;
}

</script>
